参考  
https://tjo.hatenablog.com/entry/2017/09/25/190000

In [1]:
import tensorflow as tf
import numpy as np
import csv
import pandas as pd

In [2]:
train = pd.read_csv("../input/conflict_train.csv")
test = pd.read_csv("../input/conflict_test.csv")

In [3]:
train.head()

,a1,a2,a3,a4,a5,a6,a7,cv
0,0,1,0,1,0,0,0,0
1,1,1,0,1,0,0,0,0
2,0,1,1,1,0,0,1,0
3,1,1,1,1,0,0,0,0
4,0,0,0,1,0,0,1,0


In [4]:
test.head()

,a1,a2,a3,a4,a5,a6,a7,cv
0,1,0,0,1,0,0,0,0
1,0,0,0,1,0,0,1,0
2,0,1,0,1,0,0,0,0
3,0,1,1,1,0,0,0,0
4,0,1,1,1,1,0,1,0


In [29]:
x_train = train[['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7']]
y_train = train[['cv']]
x_test = test[['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7']]
y_test = test[['cv']]

In [30]:
x = tf.placeholder(tf.float32, [None, 7])
y = tf.placeholder(tf.float32, [None, 1])
W = tf.Variable(tf.zeros([7, 1]))
b = tf.Variable(tf.zeros([1]))
tf.get_variable_scope().reuse_variables()

In [39]:
y_reg = tf.matmul(x, W) + b
y_reg_sigmoid = tf.sigmoid(y_reg)
# cross_entropy = -tf.reduce_sum(y * tf.log(y_reg_sigmoid))
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = y_reg)
cost = tf.reduce_mean(cross_entropy)
optimizer = tf.train.GradientDescentOptimizer(0.00075).minimize(cost)

In [40]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [41]:
num_epochs = 5000
for i in range(num_epochs):
    sess.run(optimizer, feed_dict={x: x_train, y: y_train})
print(sess.run(W), sess.run(b))

[[ 0.1293232 ]
 [-0.14737935]
 [-0.013942  ]
 [-0.4184805 ]
 [ 0.26410604]
 [ 0.6716626 ]
 [-0.04097668]] [-0.05328732]


In [44]:
pred2 = sess.run(y_reg_sigmoid, feed_dict={x:x_test})
pred_df = pd.DataFrame(pred2)
#pred_df = pred_df.round(1)
pred_df = pred_df.applymap(lambda x: x>=.5)
pred_df = pred_df.astype(int)

In [45]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
print(confusion_matrix(y_test, pred_df), accuracy_score(y_test, pred_df))

[[270  30]
 [ 17 283]] 0.9216666666666666
